In [4]:
import requests
import pandas 
import numpy 
import json
import streamlit as st 

In [5]:
def json_to_series(text):
    keys, values = zip(*[(dct['label'], dct['value']) for dct in json.loads(text)])
    return pd.Series(values, index=keys)


index = 1
req_total = []
df_final = pandas.DataFrame()
while index < 2:
    req_veiculos = requests.get("https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros%2Fms%3Fanunciante%3DConcession%25C3%25A1ria%257CLoja%26estadocidade%3DMato%2520Grosso%2520do%2520Sul", params="actualPage=" + str(index))
    req_veiculos = json.loads(req_veiculos.text)
    try:
        req_veiculos = req_veiculos['SearchResults']
        json_str = json.dumps(req_veiculos)
        veiculos = pandas.read_json(json_str)

        tabela_especificacoes = pandas.json_normalize(veiculos['Specification'])
        veiculos = veiculos.join(other= tabela_especificacoes)

        tabela_precos = pandas.json_normalize(veiculos['Prices'])
        veiculos = veiculos.join(other= tabela_precos)

        tabela_vendedor = pandas.json_normalize(veiculos['Seller'])
        veiculos = veiculos.join(other= tabela_vendedor)

            # tabela_atributos = pandas.json_normalize(veiculos['VehicleAttributes'])
            # veiculos = veiculos.join(other= tabela_atributos)
        try:
            veiculos = veiculos.drop(columns=['Prices', 'HotDeal', 'Channels', 'VehicleAttributes', 'Media', 'Specification', 'Seller'])
            df_final = df_final.append(veiculos, ignore_index=True)
            print("Capturado página .... {}".format(index))
        except:
            print(f"Ocorreu um erro na página {index}")
    except:
        print(f"Ocorreu um erro na página {index}")
    index +=1
    
df_final.to_csv('teste2.csv', encoding="utf8")

2021-05-19 22:49:09.533 INFO    numexpr.utils: NumExpr defaulting to 4 threads.
Ocorreu um erro na página 1


In [6]:
#df_final.to_csv('teste2.csv', encoding="utf8")

In [69]:
qtdVeiculosMarca = veiculos[['UniqueId', 'Make.Value', 'UniqueId']].groupby(by = veiculos['Make.Value'], as_index=True).agg('count')
qtdVeiculosMarca = qtdVeiculosMarca.drop(columns={'UniqueId'})
qtdVeiculosMarca = qtdVeiculosMarca.rename(columns={'Make.Value': 'Quantidade'})
st.dataframe(qtdVeiculosMarca)


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)